In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib    
import pandas as pd
import scipy as sc
import csv
import math
import random
from sklearn.model_selection import train_test_split
from scipy.sparse import isspmatrix, csc_matrix, csr_matrix
from scipy.sparse.linalg import eigsh, svds
from scipy.linalg import svd
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
import io

In [2]:
from skopt import gp_minimize
from skopt.space import Real, Integer

In [3]:
eps                  = np.finfo(float).eps

In [4]:
def createList(r1,r2):
    return [item for item in range(r1,r2 + 1)]

In [5]:
def userSim(Y): 
    print("In userSim")
    N,M              = Y.shape  
    
    Y01              = Y.copy()
    Y01[Y01>0]       = 1
    
    tmp              = (Y**2)@(Y01.T)
    tmp              = tmp + tmp.T
    tmp              = tmp - 2*(Y@(Y.T))
    
    simScore         = 1.0/(1 + np.sqrt(tmp))
    
    nominator        = Y01@Y01.T
    
    denominator      = np.sum(Y01,axis=1).reshape(-1,1)
    denominator      = np.matlib.repmat((denominator),1,N)
    denominator      = denominator + denominator.T
    denominator      = denominator - nominator
    
    B1               = nominator/denominator
    B1[np.isnan(B1)] = 0
    
    simScore         = simScore*B1
    return simScore

In [6]:
def evecs(A,nEvecs):
    npix             = A.shape[0]
    useSparse        = isspmatrix(A)
    
    dd               = 1/((np.sum(A,axis=0))+eps)
    dd               = np.sqrt(dd)

    if(useSparse):
        DD           = sc.sparse.diags(dd)
    else:
        DD           = np.diag(dd)
    
    L                = DD@A@DD
    
    if(useSparse==1):
        ss,v         = eigsh(L,nEvecs,sigma=1)
        ss           = np.flip(ss)
        V            = np.flip(v,axis=1)
    else:
        u,ss,v       = svd(L)
        V            = u[:, :nEvecs]
        
    ss               = ss.reshape(-1,1)
    ss               = ss[:nEvecs]
    
    return V,ss

In [7]:
def spectralClu(A=None,K=None): 
    # A: Affinity Matrix, Higher value -> more similar
    # K: Number of CLuster
    n                = A.shape[0]
    
    D                = np.sum(A,axis=1).reshape(-1,1)
    D[D==0]          = eps
    D                = sc.sparse.spdiags((1.0/np.sqrt(D)).T,0,n,n)
    
    L                = D@A@D
    V,evals          = evecs(L,K)
    
    Vnorm            = (np.linalg.norm(V,axis=1) + eps).reshape(-1,1)
    
    V                = V/Vnorm

    kmeans           = KMeans(n_clusters=K,max_iter=maxiter,random_state=0).fit(V)
    idx              = kmeans.predict(V)

    return idx

In [8]:
def aggregratePrediction(X,UGidx):
    n,m                  = X.shape
    K1                   = len(np.unique(UGidx))
    ctmp_d               = np.ones(n)
    ctmp                 = csc_matrix((ctmp_d,(UGidx.reshape(-1,),range(n))),shape=(K1,n))

    GFreq                = np.bincount(UGidx.reshape(-1,)).reshape(-1,1)
    XPerA                = (ctmp@X)/np.matlib.repmat((GFreq),1,m)
    
    
    return XPerA

In [9]:
def MAE(a,b):
    return np.sum(abs(a*(b!=0) - b))/np.sum(b!=0)

In [10]:
def RMSE(X,Y):
    tmp                        = ((Y - X)*(Y!=0))**2
    
    return math.sqrt(np.sum(tmp)/np.sum(Y!=0))

In [11]:
def AUC(Ypre,Yt,cutoff):
    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    auc                        = 0
    uWithPair                  = 0
    for user in range(N):
        userRating             = Yt[user,:]
        userPrediction         = Ypre[user, :]
        ratedIdx               = userRating>0
        binUserRating          = 2*(userRating[ratedIdx]>=cutoff)-1
        predForKnown           = userPrediction[ratedIdx]
        
        posIdx                 = np.where(binUserRating==1)
        posIdx                 = posIdx[0].T

        negIdx                 = np.where(binUserRating==-1)
        negIdx                 = negIdx[0]
        
        if(np.size(posIdx)!=0 and np.size(negIdx)!=0):
            negIdxGrid         = np.matlib.repmat(negIdx,len(posIdx),1)
            ttlNeg             = np.size(negIdxGrid)
            negIdxGrid         = negIdxGrid.reshape(ttlNeg,1,order='F')
            posIdxGrid         = np.matlib.repmat(posIdx.reshape(-1,1),len(negIdx),1)
            pairs              = np.concatenate((negIdxGrid,posIdxGrid),axis=1)
            pairsPred          = np.concatenate((pairs,predForKnown[negIdxGrid],predForKnown[posIdxGrid]),axis = 1)
            pairsPredWithScore = np.concatenate([pairsPred,(0.5*(pairsPred[:,2]==pairsPred[:,3]).reshape(-1,1)),(1*(pairsPred[:,2]<pairsPred[:,3]).reshape(-1,1))],axis=1)
            auc                = auc + (np.sum(pairsPredWithScore[:,4:6]))/pairsPredWithScore.shape[0]
            uWithPair          = uWithPair + 1    
    auc                        = auc/uWithPair
    
    return auc

In [12]:
def precAtK(Ypre,Yt,k,cutoff):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: precision@k

    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    prec                       = np.zeros((1,k))

    for user in range(N):
        userRating             = Yt[user, :]
        
        ratedRelevantIdx       = np.where(userRating>=cutoff)
        ratedRelevantIdx       = ratedRelevantIdx[0]

        ratedIdx               = np.where(userRating!=0)
        ratedIdx               = ratedIdx[0]

        userPrediction         = Ypre[user, :]
        prediction             = userPrediction[ratedIdx]
        sortesPIdxTmp          = np.argsort(prediction)[::-1]
        sortesPIdx             = ratedIdx[sortesPIdxTmp]
        nz                     = len(sortesPIdx)
        for kNo in range(k):
            intrsct            = np.intersect1d(sortesPIdx[0:(min(kNo,nz)+1)],ratedRelevantIdx)
            mx                 = max(min(kNo+1,nz),eps)
            prec[:,kNo]        = prec[:,kNo] + len(intrsct)/mx
            
    prec                       = prec/N
    
    return prec

In [13]:
def recallAtK(Ypre,Yt, k, cutoff):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: recall@k

    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    recall                     = np.zeros((1,k))
    
    for user in range(N):
        userRating             = Yt[user, :]
        userPrediction         = Ypre[user, :]
        
        ratedRelevantIdx       = np.where(userRating>=cutoff)
        ratedRelevantIdx       = ratedRelevantIdx[0]

        ratedIdx               = np.where(userRating!=0)
        ratedIdx               = ratedIdx[0]

        prediction             = userPrediction[ratedIdx]
        
        sortesPIdxTmp          = np.argsort(prediction)[::-1]
        sortesPIdx             = ratedIdx[sortesPIdxTmp]
        
        nz                     = len(sortesPIdx)
        for kNo in range(k):
            intrsct            = np.intersect1d(sortesPIdx[0:(min(kNo,nz)+1)],ratedRelevantIdx)
            mx                 = max(len(ratedRelevantIdx),eps)
            recall[:,kNo]      = recall[:,kNo] + len(intrsct)/mx
            
    recall                     = recall/N
    
    return recall 

In [14]:
def ndcgAtk(Ypre,Yt,k):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: precision@k

    res                        = np.zeros((1,k))
    cnt                        = 0
    N,M                        = Yt.shape

    for user in range(N):
        ratedIdx               = Yt[user,:]!=0

        userRating             = Yt[user,ratedIdx]
        userPrediction         = Ypre[user,ratedIdx]
        nz                     = len(userRating)

        ranks                  = np.zeros((1,nz))
        ideal_ranks            = np.zeros((1,nz))
        
        I                      = np.argsort(-userPrediction,axis=0)
        ideal_I                = np.argsort(-userRating,axis=0)

        ranks                  = I
        ideal_ranks            = ideal_I

        oriOrder               = np.argsort(ideal_ranks,axis=0)

        nominator              = userRating/(np.log(ranks+2))
        denominator            = userRating/(np.log(ideal_ranks + 2))
    
        nominator              = nominator[oriOrder]
        denominator            = denominator[oriOrder]
        
        if k > nz:
            nominator          = np.concatenate((nominator, np.zeros((k - nz))))
            denominator        = np.concatenate((denominator, np.zeros((k - nz))))
        elif k < nz:
            nominator          = nominator[0:k]
            denominator        = denominator[0:k]          

        if np.array(np.where(np.cumsum(denominator)== 0)).shape[1] != 0:
            tmp                = np.zeros((1,k))
        else:
            tmp                = np.cumsum(nominator) / np.cumsum(denominator)
            cnt                = cnt + 1
        
        res                    = res + tmp
        
    res                        = res/cnt
    
    return res

In [15]:
def EvaluationAllUpdated(Yprd,Y1,k,cutoff):
    mae                        = MAE(Yprd, Y1) 
    rmse                       = RMSE(Yprd, Y1)
    auc                        = AUC(Yprd, Y1, cutoff)
    precision                  = precAtK(Yprd,Y1, k, cutoff)
    recall                     = recallAtK(Yprd,Y1, k, cutoff)
    f1                         = (2*precision*recall)/(precision + recall + eps)
    ndcg                       = ndcgAtk(Yprd,Y1,k)

    return mae,rmse,auc,precision,recall,f1,ndcg

In [16]:
def EvaluationAllUpdated_N(Yprd,Y1,k,cutoff):
    mae                        = MAE(Yprd, Y1) 
    rmse                       = RMSE(Yprd, Y1)

    return mae,rmse

In [17]:
cols                     = ["Model","MAE","RMSE","AUC","Precision@1","Precision@5","Precision@10","Precision@20",
                            "Precision@30","Precision@40","Recall@1","Recall@5","Recall@10","Recall@20","Recall@30",
                            "Recall@40","F1@1","F1@5","F1@10","F1@20","F1@30","F1@40","NDCG@1","NDCG@5","NDCG@10",
                            "NDCG@20","NDCG@30","NDCG@40"]
Result_Trn               = pd.DataFrame(columns=cols)

In [18]:
cols                     = ["Model","MAE","RMSE"]
Result_Trn_CDR               = pd.DataFrame(columns=cols)
Result_Tst_CDR = pd.DataFrame(columns=cols)

In [19]:
def gradUnifiedLS(v,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx): 
    
    n,m                  = Y.shape
    U                    = v[0:n*d].reshape(n,d,order='F')
    V                    = v[n*d:n*d+m*d].reshape(m,d,order='F')
    UG                   = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
    VG                   = v[n*d+m*d+K1*d:].reshape(K2,d,order='F')

    UGmatMU              = (UG[UGidx.reshape(-1,),:]) - U
    VGmatMV              = (VG[VGidx.reshape(-1,),:]) - V
    
    X                    = U@(V.T)
    Ygt0                 = Y>0
    YMXgt0               = (Y - X)*Ygt0

    regobj               = (lambda3/2)*(np.sum(U**2) + np.sum(V**2))

    lossobj              = 0
    lossobj              = lossobj + (0.5*(np.sum(YMXgt0**2)))
    lossobj              = lossobj + (lambda1/2)*(np.sum(UGmatMU**2) + np.sum(VGmatMV**2))
    
    dU                   = lambda3*U
    dV                   = lambda3*V
    
    dU                   = dU - YMXgt0@V
    dV                   = dV - (YMXgt0.T)@U

    dU                   = dU - (lambda1*UGmatMU)
    dV                   = dV - (lambda1*VGmatMV)

    ctmp_d               = np.ones(n)                   # convert UGidx to a K1-by-n matrix containing the k1 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(UGidx.reshape(-1,),range(n))),shape=(K1,n))
    dUG                  = lambda1*((ctmp)@UGmatMU)
    
    ctmp_d               = np.ones(m)                   # convert UGidx to a K2-by-m matrix containing the k2 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(VGidx.reshape(-1,),range(m))),shape=(K2,m))
    dVG                  = lambda1*((ctmp)@VGmatMV)
    
    obj                  = regobj + lossobj
   
    grad                 = np.concatenate((dU.flatten('F'),dV.flatten('F'),dUG.flatten('F'),dVG.flatten('F'))).reshape(-1,1)
    
    return obj,grad,lossobj,regobj

In [20]:
def gradUnifiedMMMF(v,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGIdx,VGIdx):

    n,m                  = Y.shape
    U                    = v[0:n*d].reshape(n,d,order='F')
    V                    = v[n*d:n*d+m*d].reshape(m,d,order='F')
    theta                = v[n*d+m*d:n*d+m*d+n*(l-1)].reshape(n,l-1,order='F')
    UG                   = v[n*d+m*d+n*(l-1):n*d+m*d+n*(l-1)+K1*d].reshape((K1,d), order='F')
    VG                   = v[n*d+m*d+n*(l-1)+K1*d:n*d+m*d+n*(l-1)+K1*d+K2*d].reshape((K2,d), order='F')
    thetaG               = v[n*d+m*d+n*(l-1)+K1*d+K2*d:].reshape((K1,l-1), order='F')
        
    UGmatMU              = (UG[UGIdx.reshape(-1,),:]) - U
    VGmatMV              = (VG[VGIdx.reshape(-1,),:]) - V
    thetaGMtheta         = (thetaG[UGIdx.reshape(-1,),:]) - theta
    
    X                    = U@V.T
    Ygt0                 = Y>0
    BX                   = X*Ygt0
    
    dU                   = lambda3*U
    dV                   = lambda3*V
    dtheta               = np.zeros((n, l-1))
    
    regobj               = (lambda3/2)*(np.sum(U**2) + np.sum(V**2))
    
    lossobj              = 0
    lossobj              = lossobj + (lambda1/2)*(np.sum(UGmatMU**2) + np.sum(VGmatMV**2) + np.sum(thetaGMtheta**2))
    
    for k in range(l-1):
        S                = Ygt0 - 2*(Y>k+1)
        BZ               = (theta[:,k].reshape(-1,1)@(np.ones([1,m])))*S - BX*S
        lossobj          = lossobj + sum(sum(h(BZ)))
        tmp              = hprime(BZ)*S
        dU               = dU - tmp@V
        dV               = dV - tmp.T@U
        dtheta[:,k]      = tmp@np.ones((m,))
        
    dU                   = dU - lambda1*UGmatMU
    dV                   = dV - lambda1*VGmatMV
    dtheta               = dtheta - lambda1*thetaGMtheta
    
    ctmp_d               = np.ones(n)                   # convert UGidx to a K1-by-n matrix containing the k1 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(UGIdx.reshape(-1,),range(n))),shape=(K1,n))
    dUG                  = np.multiply(lambda1,(ctmp@UGmatMU))
    dthetaG              = np.multiply(lambda1,(ctmp@thetaGMtheta))
 
    ctmp_d               = np.ones(m)                   # convert UGidx to a K2-by-m matrix containing the k2 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(VGIdx.reshape(-1,),range(m))),shape=(K2,m))
    dVG                  = np.multiply(lambda1,(ctmp@VGmatMV))
    
    obj                  = regobj + lossobj;            # obj is the objective function that we need to minimize

    grad                 = np.concatenate((dU.flatten('F'),dV.flatten('F'),dtheta.flatten('F'),dUG.flatten('F'),dVG.flatten('F'),dthetaG.flatten('F'))).reshape(-1,1)
    
    return obj,grad,lossobj,regobj

In [21]:
def conjgrad(x0,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx):
    J                     = []
    temp                  = 0
    nu                    = 0.1
    abstol                = 0                                 # stop if gradient magnitude goes below this
    allowNonDecrease      = 0                                 # don't stop if line search fails to find decrease
    digits                = 12                                # digits of precision to use for objective comparisons
    ogfun                 = objGrad
    ogcalls               = 0
    x                     = x0
    numiter               = 0
    j                     = 0
    alpha                 = 10**(-10)
    ogcalls               = ogcalls + 1
    obj,dx,lossobj,regobj = ogfun(x,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
    r                     = -dx
    s                     = r
    dirn                  = s
    deltanew              = (r.T)@dirn
    deltazero             = deltanew
    
    while ((numiter<maxiter) and (abs(deltanew)>tol*tol*abs(deltazero)) and (abs(deltanew)>abstol)):
        numiter           = numiter + 1
        j                 = j + 1
        print('\n %.4f' %obj)
        J.append(obj)
        prevobj           = obj

        if (alpha < 10**(-10)):
            alpha         = 10**(-10)

        alpha,obj,dx,ogc  = cgLineSearch(x,obj,dx,dirn,alpha,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,lossobj,regobj)
        ogcalls           = ogcalls + ogc
        
        temp              = temp + alpha
        x                 = x + alpha*dirn
        r                 = -dx
        deltaold          = deltanew
        deltamid          = r.T@s
        deltanew          = r.T@r
        beta              = (deltanew - deltamid) / deltaold
        dirn              = r + max(0, beta)*dirn
        if ((deltamid/deltanew >= nu) or (dirn.T@dx >= 0)):
            dirn          = r
            j             = 0
        s                 = r
        print(numiter)
        
    return x,numiter,ogcalls,J

In [22]:
def cgLineSearch(x0,obj0,dx0,direction,alpha,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,lobj,robj):
    seciter                   = 5                               # maximum number of quadratic interpolation iterations
    alpha0                    = alpha
    c1                        = 10**(-4)                        # required decrease in objective (relative to gradient)
    digits                    = 12                              # digits of precision to use for objective comparisons
    gamma                     = 10
    ogfun                     = objGrad
    
    if (alpha0 <= 0):                                           # check initial values
        print('alpha0 must be greater than zero')
    
    obj                       = obj0                            # begin line search
    dx                        = dx0
    etazero                   = (dx.T)@direction
    etaprev                   = etazero
    alpha                     = alpha0
    ogcalls                   = 0
    lossobj                   = lobj
    regobj                    = robj
    
    alpha,obj,dx,lossobj,regobj,ogcalls = findNonZeroAlpha(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj)
    
    obj,dx,lossobj,regobj     = ogfun(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
    ogcalls                   = ogcalls + 1
    
    oldalpha,oldobj,olddx                           = saveAlpha(alpha,obj,dx)
    alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack = backtrack(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx)

    beta                      = alpha
    eta                       = dx.T@direction
    i                         = 0
    
    while((abs(eta)>c2*abs(etazero)) and (i<seciter) and (pround(obj,digits) <= pround(obj0,digits)) and (etaprev!=eta) and (np.sum((x0 + alpha*direction) != x0)>0)):
        beta                  = eta*beta / (etaprev - eta)
        oldalpha,oldobj,olddx = saveAlpha(alpha,obj,dx)
        alpha                 = alpha + beta
        if(alpha<=0):
            alpha             = 1
        etaprev               = eta
        i                     = i + 1
        obj,dx,lossobj,regobj = ogfun(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        eta                   = dx.T@direction

    alpha,obj,dx,lossobj,regobj,ogcalls             = findNonZeroAlpha(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj)
    alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack = backtrack(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx)
    checkConditions(obj,digits,obj0,etazero,eta,x0,alpha,direction)
    
    return alpha,obj,dx,ogcalls 

In [23]:
def findNonZeroAlpha(x0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj):
    if(np.array_equal((x0+alpha*direction),x0)):                # Make sure alpha isn't smaller than level of precision
        preAlpha              = alpha
        
        while(np.array_equal((x0+alpha*direction),x0)):
            alpha             = alpha*gamma
        
        obj,dx,lossobj,regobj = objGrad(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        
    return alpha,obj,dx,lossobj,regobj,ogcalls

In [24]:
def backtrack(x0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx):
    #ensures either (1) Armijo lowex0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcallsr objective, or (2) infinitesimal alpha
    doBacktrack               = 0
    preAlpha                  = alpha
    while((pround(obj,digits)>pround(obj0 + c1*alpha*etazero, digits)) and (np.sum((x0+alpha*direction)!=x0)>0)):
        if(ogcalls > 1):
            if((oldalpha > alpha/gamma) and (oldobj < (obj0+c1*oldalpha*etazero))):
                alpha,obj,dx  = restoreAlpha(oldalpha,oldobj,olddx)
                doBacktrack   = 1
                break
        alpha                 = alpha/gamma
        obj,dx,lossobj,regobj = objGrad(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        doBacktrack           = 1
    return alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack

In [25]:
def saveAlpha(alpha,obj,dx):
    #make sure current step yields decrease in objective
    oldalpha   = alpha
    oldobj     = obj
    olddx      = dx
    return oldalpha,oldobj,olddx

In [26]:
def restoreAlpha(oldalpha,oldobj,olddx):
    alpha      = oldalpha
    obj        = oldobj
    dx         = olddx
    return alpha,obj,dx

In [27]:
def pround(x,d):
    d          = round(d)
    if (d<1):
        print('Number of digits must be integer d = %.4e \n'%d)
    if(x==0 or math.isnan(x) or math.isinf(x)):
        y      = x   
    else:
        p      = math.floor(math.log10(abs(x)))+1
        factor = 10**(d-p)
        y      = np.round(x*factor)/factor
    return y

In [28]:
def checkConditions(obj,digits,obj0,etazero,eta,x0,alpha,direction):
    if round(obj, digits) >= round(obj0, digits):
        print('Warning: Finished line search without decreasing objective.\nMay have reached limit of precision, or obj/grad code may be broken.\n')
    if etazero == eta:
        print('Warning: Line search yielded no change in directional derivative.\nMay have reached limit of precision.\n')
    if sum(x0 + alpha * direction != x0) == 0:
        print('Warning: Line search yielded no change in position.\nMay have reached limit of precision.\n')

In [29]:
def h(z):
    zin01      = (z>0)^(z>=1)
    zle0       = z<0
    ret        = zin01/2 - zin01*z + zin01*(z**2)/2 + zle0/2 - zle0*z
    return ret  

In [30]:
def hprime(z):
    zin01      = (z>0)^(z>=1)
    zle0       = z<0
    ret        = zin01*z - zin01 - zle0
    return ret

In [31]:
def m3fSoftmax(xy,theta):
    n,m        = xy.shape
    n1,l1      = theta.shape
    if(n!=n1):
        print('sizes of xy and theta don''t match');
    y          = np.ones((n,m))
    for i in range(l1):
        tmp    = ((theta[:,i]).reshape(-1,1))@(np.ones((1,m)))
        tmp    = xy>=tmp
        y      = y + tmp
    return y

## Loading Matrices for Video Games domain

In [32]:
from pandas import read_csv

In [33]:
rating_matrix_video = np.loadtxt("rating_matrix_video_2k.csv", delimiter=",")

In [34]:
temp_d = read_csv('new_user_ids_video_2k.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
new_user_ids_video_2k = temp_np.reshape(temp_np.shape[0],)

In [35]:
temp_d = read_csv('item_ids_video_2k_rated_unrated.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
item_ids_video_2k_rated_unrated = temp_np.reshape(temp_np.shape[0],)

## Loading Matrics for Movies and TV domain

In [290]:
rating_matrix_mt = np.loadtxt("rating_matrix_mt_2k.csv", delimiter=",")

In [291]:
temp_d = read_csv('new_user_ids_mt_2k.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
new_user_ids_mt_2k = temp_np.reshape(temp_np.shape[0],)

In [292]:
temp_d = read_csv('item_ids_mt_2k_rated_unrated.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
item_ids_mt_2k_rated_unrated = temp_np.reshape(temp_np.shape[0],)

## Removing overlapping test user_ids from target domain

In [293]:
d                        = 100 

In [294]:
overlapping_user_ids_t = np.intersect1d(new_user_ids_video_2k, new_user_ids_mt_2k)
print(overlapping_user_ids_t)
print(overlapping_user_ids_t.shape)

['A0002090WKEMAO8KOWKM' 'A00230923E4Y7VHWZK0IC' 'A002439424KGHR3LZ1OMZ'
 ... 'A11MLVQFZ4EA2R' 'A11MSQJV1W2W6D' 'A11MV1WAIEYKK0']
(2000,)


In [295]:
np.random.seed(15)
train = np.random.choice(overlapping_user_ids_t, size=math.ceil(0.8*overlapping_user_ids_t.shape[0]), replace=False)
test = np.array(list(set(overlapping_user_ids_t.tolist()).difference(set(train.tolist()))))

In [296]:
X_train = np.empty((0,100))
Y_train = np.empty((0,100))
X_test = np.empty((0,100))
Y_test = []

In [297]:
for user_id in test:
    ind1 = np.where(new_user_ids_mt_2k == user_id)
    Y_test.append(rating_matrix_mt[ind1][0])
    new_user_ids_mt_2k =np.delete(new_user_ids_mt_2k, ind1, 0)
    rating_matrix_mt = np.delete(rating_matrix_mt, ind1, 0)

In [298]:
Y_test = np.array(Y_test)
print(Y_test.shape)

(400, 11480)


In [299]:
print(new_user_ids_mt_2k.shape)
print(rating_matrix_mt.shape)

(6600,)
(6600, 11480)


## GRS for Video Games domain

In [46]:
epochs                   = 3
tstPer                   = 30                               # Testing Percentage
NoTstIFromG              = 50
l                        = 5                                # Rating Level

K1                       = 10                              #Number of Clusters in User Space
K2                       = 22                              #Number of Clusters in Item Space

d                        = 100                              # latent space size
minUserPerForSel         = 20                               #Item will be select for test only if minUserPerForSel percentage of users in a group has rated
k                        = 40                               # eval@k: precision@k, recall@k
cutoff                   = 3                                # Relevant > cutoff

lambda1UnifiedLS         = 1
lambda3UnifiedLS         = 1

c2                       = 10**-2
tol                      = 10**-3;
maxiter                  = 500;

In [47]:
n, m = rating_matrix_video.shape
print(n, m)

9000 7500


In [48]:
#Spectral Clustering
simU                     = userSim(rating_matrix_video)
simU                     = (simU + simU.T)/2
UGidx_video              = spectralClu(simU,K1).reshape(-1,1)

simI                     = userSim(rating_matrix_video.T)
simI                     = (simI + simI.T)/2
VGidx_video              = spectralClu(simI,K2).reshape(-1,1)

In userSim
In userSim


In [49]:
print(UGidx_video)
print(UGidx_video.shape)

[[2]
 [2]
 [0]
 ...
 [0]
 [0]
 [0]]
(9000, 1)


In [50]:
print(VGidx_video)
print(VGidx_video.shape)

[[ 3]
 [ 0]
 [19]
 ...
 [ 0]
 [15]
 [ 0]]
(7500, 1)


In [51]:
unique, frequency = np.unique(UGidx_video, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

print()

unique, frequency = np.unique(VGidx_video, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0 1 2 3 4 5 6 7 8 9]
Frequency Values: [5417  365  447  340  384  417  446  418  387  379]

Unique Values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
Frequency Values: [4059  193  188  208  172  182  172  169  118  170  128  171  114  175
  155  158  171  177  168  140  157  155]


### Recommendation

In [52]:
np.random.seed(0)
vini = np.random.randn((n*d + m*d + K1*d + K2*d), 1)

In [53]:
# Group Recommendation: Unified LS
objGrad             = gradUnifiedLS
v,numiter,ogcalls,J = conjgrad(vini,objGrad,c2,tol,maxiter,l,d,rating_matrix_video,lambda1UnifiedLS,lambda3UnifiedLS,K1,K2,UGidx_video,VGidx_video)
U_video                   = v[0:n*d].reshape(n,d,order='F')
V_video                   = v[n*d:n*d+m*d].reshape(m,d,order='F')
UG_video                  = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
X                   = U_video@V_video.T
X[X<=1]             = 1
X[X>=5]             = 5
XPerA               = aggregratePrediction(X,UGidx_video)
XPerAMat            = XPerA[UGidx_video.reshape(-1,),:]

XPer_Grp                    = UG_video@V_video.T
XPer_GrpMat                 = XPer_Grp[UGidx_video.reshape(-1,),:]
XPer_GrpMat[XPer_GrpMat<=1] = 1
XPer_GrpMat[XPer_GrpMat>=5] = 5


 3392678.0584
1

 2762934.2746
2

 2391850.8186
3

 1709290.2181
4

 1629484.6525
5

 1594679.9195
6

 1585929.9877
7

 1554435.1939
8

 1537235.5801
9

 1466090.5826
10

 1444152.6314
11

 1312878.8553
12

 1216491.8557
13

 996828.4700
14

 776131.1016
15

 649849.2303
16

 501742.7163
17

 453694.5856
18

 406258.8340
19

 382762.8783
20

 362314.1889
21

 343159.9345
22

 320902.4055
23

 309911.6991
24

 297796.6283
25

 282378.2591
26

 269100.8352
27

 261836.1682
28

 252021.9490
29

 247474.4476
30

 242124.3967
31

 235885.4213
32

 231494.1466
33

 226191.2721
34

 220028.3257
35

 213657.1070
36

 206462.5041
37

 197427.7197
38

 185109.1208
39

 174661.4446
40

 167573.7997
41

 144440.1340
42

 134478.8185
43

 123252.4929
44

 121844.0125
45

 116867.8070
46

 115883.6545
47

 105764.6616
48

 104550.5639
49

 93500.3773
50

 91973.6577
51

 86047.0260
52

 85061.4522
53

 79686.0063
54

 78568.7045
55

 75771.2584
56

 72562.8643
57

 70114.9262
58

 67886.2154
59

 6

In [54]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPer_GrpMat, rating_matrix_video, k, cutoff)
ResultTrnULS_Grp                      = ["Unified_LS_Grp",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Video Games domain - Run 1",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

## GRS for Movies and TV domain

In [300]:
epochs                   = 3
tstPer                   = 30                               # Testing Percentage
NoTstIFromG              = 50
l                        = 5                                # Rating Level

K1                       = 10                               #Number of Clusters in User Space
K2                       = 100                             #Number of Clusters in Item Space

d                        = 100                              # latent space size
minUserPerForSel         = 20                               #Item will be select for test only if minUserPerForSel percentage of users in a group has rated
k                        = 40                               # eval@k: precision@k, recall@k
cutoff                   = 3                                # Relevant > cutoff

lambda1UnifiedLS         = 30
lambda3UnifiedLS         = 1

c2                       = 10**-2
tol                      = 10**-3;
maxiter                  = 500;

In [301]:
n, m = rating_matrix_mt.shape
print(n, m)

6600 11480


In [302]:
#Spectral Clustering
simU                     = userSim(rating_matrix_mt)
simU                     = (simU + simU.T)/2

UGidx_mt                    = spectralClu(simU,K1).reshape(-1,1)

simI                     = userSim(rating_matrix_mt.T)
simI                     = (simI + simI.T)/2

VGidx_mt                    = spectralClu(simI,K2).reshape(-1,1)

In userSim
In userSim


C:\Users\NIT\AppData\Local\Temp\ipykernel_14692\3829436230.py:21: RuntimeWarning: invalid value encountered in true_divide
  B1               = nominator/denominator


In [303]:
print(UGidx_mt)
print(UGidx_mt.shape)

[[1]
 [1]
 [9]
 ...
 [4]
 [4]
 [1]]
(6600, 1)


In [304]:
print(VGidx_mt)
print(VGidx_mt.shape)

[[3]
 [1]
 [1]
 ...
 [1]
 [1]
 [3]]
(11480, 1)


In [305]:
unique, frequency = np.unique(UGidx_mt, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

print()

unique, frequency = np.unique(VGidx_mt, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0 1 2 3 4 5 6 7 8 9]
Frequency Values: [ 349 3746  302  323  291  349  293  311  351  285]

Unique Values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Frequency Values: [  21 7154   28  742    1   27   20   27    5   28   41    9   18   25
   25    4  684   29   16    9   18   24   11    6   13    5  193   18
    4    3    9   20    6   35   29   37   29    3    4    5    5    4
   12    5   46   12    6   26    3    6   41    3    2    4   27    6
   47   29    3    7    4   23   27    4   25    2    6    2   29  555
    5    4    5    6   48    3   10  274   27   24    9    5    9    4
    3    5    6    5  543   28   19   39    2    8    9    4    5    6
    5    4]


### Recommendation

In [308]:
np.random.seed(15)
vini = np.random.randn((n*d + m*d + K1*d + K2*d), 1)

In [309]:
# Group Recommendation: Unified LS
objGrad             = gradUnifiedLS
v,numiter,ogcalls,J = conjgrad(vini,objGrad,c2,tol,maxiter,l,d,rating_matrix_mt,lambda1UnifiedLS,lambda3UnifiedLS,K1,K2,UGidx_mt,VGidx_mt)
U_mt                  = v[0:n*d].reshape(n,d,order='F')
V_mt                   = v[n*d:n*d+m*d].reshape(m,d,order='F')
UG_mt                  = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
X                   = U_mt@V_mt.T
X[X<=1]             = 1
X[X>=5]             = 5
XPerA               = aggregratePrediction(X,UGidx_mt)
XPerAMat_mt            = XPerA[UGidx_mt.reshape(-1,),:]

XPer_Grp                    = UG_mt@V_mt.T
XPer_GrpMat_mt                 = XPer_Grp[UGidx_mt.reshape(-1,),:]
XPer_GrpMat_mt[XPer_GrpMat_mt<=1] = 1
XPer_GrpMat_mt[XPer_GrpMat_mt>=5] = 5


 53532641.5302
1

 39640506.1836
2

 36512056.9624
3

 31401260.1024
4

 30170077.4226
5

 29272822.2843
6

 26717336.4090
7

 25496621.8453
8

 25195997.2212
9

 24409064.9307
10

 24030287.0741
11

 21955155.7458
12

 21497152.6642
13

 16108421.4005
14

 14902407.4171
15

 14358800.5778
16

 14151599.6469
17

 10855522.5108
18

 10294940.6563
19

 6796769.2018
20

 5780337.6239
21

 5552855.6620
22

 5481496.2518
23

 4987043.3291
24

 4793605.5358
25

 3778182.1695
26

 3436138.3242
27

 3049104.5257
28

 2914882.4961
29

 2696200.9049
30

 2669342.5305
31

 2378330.6275
32

 2312257.0578
33

 2114779.3047
34

 1918900.0973
35

 1680161.8969
36

 1671661.7081
37

 1591276.4386
38

 1562942.3347
39

 1507888.5179
40

 1377037.5746
41

 1224201.5567
42

 1210377.6430
43

 1182463.2001
44

 1173906.0928
45

 1153414.7828
46

 1069501.2865
47

 997962.9436
48

 980561.3381
49

 923741.6212
50

 918591.9209
51

 879403.9156
52

 865706.8607
53

 821878.8782
54

 794304.7869
55

 673981

In [269]:
# Result_Trn.drop(index=Result_Trn.index[-1],axis=0,inplace=True)

In [310]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPer_GrpMat_mt, rating_matrix_mt, k, cutoff)
ResultTrnULS_Grp                      = ["Unified_LS_Grp_mt",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Movies & TV domain - Run 3.2",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

In [311]:
Result_Trn

,Model,MAE,RMSE,AUC,Precision@1,Precision@5,Precision@10,Precision@20,Precision@30,Precision@40,...,F1@10,F1@20,F1@30,F1@40,NDCG@1,NDCG@5,NDCG@10,NDCG@20,NDCG@30,NDCG@40
0,Video Games domain - Run 1,1.378286,1.587275,0.845247,0.825222,0.808576,0.807342,0.806778,0.806630,0.806615,...,0.818304,0.819311,0.819491,0.819544,0.949633,0.991605,0.995516,0.996840,0.997110,0.997181
1,Movies & TV domain - Run 3,1.237896,1.572646,0.867577,0.896818,0.874558,0.871491,0.870317,0.870007,0.869799,...,0.880449,0.883727,0.884641,0.884993,0.914726,0.980673,0.990302,0.994341,0.995515,0.995980
2,Movies & TV domain - Run 3.1,1.404730,1.704081,0.756528,0.887576,0.871775,0.869906,0.869100,0.868835,0.868711,...,0.879139,0.882850,0.883753,0.884161,0.910365,0.979302,0.988500,0.992848,0.993992,0.994443
3,Movies & TV domain - Run 3.2,1.343549,1.946782,0.819294,0.890909,0.874510,0.871174,0.869683,0.869362,0.869154,...,0.879814,0.883154,0.884114,0.884483,0.918945,0.978702,0.988339,0.993027,0.994050,0.994530


## Creating X, Y matrices for mapping

In [312]:
#Adding individual latent vectors
for user_id in train:
    source_ind = np.where(new_user_ids_video_2k == user_id)
    target_ind = np.where(new_user_ids_mt_2k == user_id)
    X_train = np.concatenate((X_train, U_video[source_ind]), axis = 0)
    Y_train = np.concatenate((Y_train, U_mt[target_ind]), axis = 0)
    
for user_id in test:
    source_ind = np.where(new_user_ids_video_2k == user_id)
    target_ind = np.where(new_user_ids_mt_2k == user_id)
    X_test = np.concatenate((X_test, U_video[source_ind]), axis = 0)  

    
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1600, 100)
(1600, 100)
(400, 100)
(400, 11480)


In [313]:
groupsToMembers_video = []
groupsToMembers_mt = []
for i in range(K1):
    groupsToMembers_video.append([])
    groupsToMembers_mt.append([])

for i in range(UGidx_video.shape[0]):
    groupsToMembers_video[int(UGidx_video[i][0])].append(new_user_ids_video_2k[i])
    
for i in range(UGidx_mt.shape[0]):
    groupsToMembers_mt[int(UGidx_mt[i][0])].append(new_user_ids_mt_2k[i])
    
print(len(groupsToMembers_video))
print(len(groupsToMembers_mt))

10
10


In [314]:
for i in range(len(groupsToMembers_video)):
    print(len(groupsToMembers_video[i]))

5417
365
447
340
384
417
446
418
387
379


In [315]:
for i in range(len(groupsToMembers_mt)):
    print(len(groupsToMembers_mt[i]))

349
3746
302
323
291
349
293
311
351
285


In [316]:
print(groupsToMembers_mt[5])

['A2NQMXWT7SS10E', 'A3OQX196H43Y2A', 'A2DDLPZAX86JD7', 'AUDMZTG223XQI', 'A1X1LAUCO816XC', 'ARHWDGGCO0L6P', 'A10O9QQ668ZC4L', 'A37KUO7HVAIYVJ', 'A1U6T2P6H9GV9R', 'AF60OT1BRTKDP', 'A4YDPOCTQ7TQA', 'ADHJO71YC1CC7', 'A2TW0S41ZSX2X9', 'A32PFGGZNWJSO1', 'A8O3S7HN6JU1E', 'A1160SDYMK6678', 'A3A0W2FYI5PQA1', 'A12GD7YRAGLMP4', 'AY8EESCPLFHCH', 'A1XJZO4W3HX7SP', 'A3MJCXQVVTRSR9', 'A1X84SI3JROQME', 'A1QDG6ACW3CUSN', 'A3J1UYCB7WOYXI', 'A11K1VOFOB1VRO', 'AHEEJ1OTE08A', 'AJ3CX3XU4UNNH', 'A1Y5OA6PK791Z8', 'AJUKLJLVR7HY', 'A3RV6XHEZL53VG', 'A2XZ0PUHT2661E', 'A3OGGAKG5WHXFY', 'A117B3QAGUDV1O', 'A109IH3ZKH2TMQ', 'A2AZK5L7W8YNSI', 'A11LRUPBEBJDNY', 'A11I9559P340QU', 'A3RWEWJID5KXTD', 'A100STFODFKOHW', 'A3DPYDKTYQM91S', 'A1B9Y5RX4I0Y6G', 'A3KXIIPH6D8N10', 'A2M08S3CMA0RQJ', 'A33E2GMNIZ1SAF', 'A1080VOU32OCLY', 'A39IG2RVDH54I2', 'A103ILT4IKON4K', 'A3FAGO8XMU1LUW', 'A3UCJZQGA8CICE', 'AUI84VBUBSU7H', 'A3FNZ5764CSF32', 'ASTJ4QAR9YPM4', 'AQ1DOQMPQXKZD', 'A9I06FO4U8FEF', 'A2WKNYBFOOKLX2', 'A10I5VZUOO5OME', 'A32NZD

In [317]:
counter = 0
for i in range(len(groupsToMembers_video)):
    currGroup_video = groupsToMembers_video[i]
    for j in range(len(groupsToMembers_mt)):
        currGroup_mt = groupsToMembers_mt[j]
        if len(set(currGroup_video).intersection(set(currGroup_mt))) > 0:
            counter = counter + 1
            X_train = np.row_stack((X_train, UG_video[i]))
            Y_train = np.row_stack((Y_train, UG_mt[j]))
            
print(X_train.shape)
print(Y_train.shape)
print(counter)

(1694, 100)
(1694, 100)
94


## MLP Mapping

In [318]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

In [319]:
class GetDataset(torch.utils.data.Dataset):

  def __init__(self, X, y, scale_data=True):
    if not torch.is_tensor(X) and not torch.is_tensor(y):
      self.X = torch.from_numpy(X)
      self.y = torch.from_numpy(y)

  def __len__(self):
      return len(self.X)

  def __getitem__(self, i):
      return self.X[i], self.y[i]

In [320]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(100, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 100)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [321]:
# Set fixed random number seed
torch.manual_seed(43)

In [322]:
dataset = GetDataset(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=X_train.shape[0], shuffle=False, num_workers=0)

In [323]:
# space = [
#     Real(1e-3, 5e-2, name='lr'),
#     Real(1e-5, 1e-2, name='decay')
# ]

In [324]:
# def trainMLPAndGetMAE(hyperparameters):
#     # Extract hyperparameters
#     learning_rate = hyperparameters[0]
#     wt_decay = hyperparameters[1]
    
#     mlp = MLP()
  
#     # Define the loss function and optimizer
#     loss_function = nn.MSELoss()
#     optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate, weight_decay=wt_decay)
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, verbose=True)
    
#     counter = 0
#     # Run the training loop
#     while optimizer.param_groups[-1]['lr'] > 1e-4: # 5 epochs at maximum

#         # Print epoch
#     #     print(f'Starting epoch {epoch+1}')

#         # Set current loss value
#         current_loss = 0.0
#         losses = []
#         # Iterate over the DataLoader for training data
#         for i, data in enumerate(trainloader, 0):

#           # Get and prepare inputs
#     #         if counter > 2:
#     #             break
#     #         print(counter)
#             inputs, targets = data
#             inputs, targets = inputs.float(), targets.float()
#             targets = targets.reshape((targets.shape[0], 100))

#     #         print("Inputs shape: ", inputs.shape)
#     #         print("Target shape: ", targets.shape)
#     #         counter += 1

#           # Zero the gradients
#             optimizer.zero_grad()

#           # Perform forward pass
#             outputs = mlp(inputs)
#     #         print("Output shape: ", outputs.shape)

# #             outputs_np = outputs.detach().numpy()
# #             targets_np = targets.detach().numpy()
#             t_V_mt = torch.from_numpy(V_mt.T).float()

# #             t_vec = targets_np@V_mt.T
# #             o_vec = outputs_np@V_mt.T
#             t_vec = torch.matmul(targets, t_V_mt)
#             o_vec = torch.matmul(outputs, t_V_mt)
#     #         print("t_vec.shape: ", t_vec.shape)
#     #         print("o_vec.shape: ", o_vec.shape)
#             indicator_matrix = (t_vec >= 0.7)
#     #         print("indicator matrix: ", indicator_matrix)
#     #         print(np.all(indicator_matrix == True))
# #             t_vec_interacted = torch.from_numpy(t_vec*indicator_matrix).requires_grad_() 
# #             o_vec_interacted = torch.from_numpy(o_vec*indicator_matrix).requires_grad_() 
#             t_vec_interacted = torch.mul(t_vec, indicator_matrix)
#             o_vec_interacted = torch.mul(o_vec, indicator_matrix)
# #             t = torch.from_numpy(a)

#           # Compute loss
#             loss = loss_function(o_vec_interacted, t_vec_interacted)
#     #         print("Loss: ", loss.item())
#             print("Epoch : " + str(counter + 1) + " | Error : " + str(loss.item()) + " | LR : " + str(optimizer.param_groups[-1]['lr']))
#             losses.append(loss.item())
#           # Perform backward pass
#             loss.backward()

#           # Perform optimization
#             optimizer.step()


#         mean_loss = sum(losses) / len(losses)
#         scheduler.step(mean_loss)
# #         print(f"Loss at epoch {counter + 1} = {mean_loss}")
#         if optimizer.param_groups[-1]['lr'] <= 1e-4:
#             break;
#         counter += 1

#     # Process is complete.
#     print('Training process has finished.')
    
#     pred_X_train = mlp(torch.from_numpy(np.float32(X_train))).detach().numpy()
#     print("pred_X_train: ", pred_X_train.shape)
#     par1 = pred_X_train@V_mt.T
#     par2 = Y_train@V_mt.T
#     mae = MAE(par1, par2)
#     print("MAE: ", mae)
#     print()
#     print()
#     return mae

In [325]:
# def objective(hyperparameters):
#     print(hyperparameters)
#     return trainMLPAndGetMAE(hyperparameters)

In [326]:
# # Perform Bayesian optimization
# result = gp_minimize(objective, space, n_calls=20)

In [327]:
# # Get the best hyperparameters and performance
# # best_hyperparameters = {
# #     'K1' = result.x[0]
# #     'K2' = result.x[1]
# #     'lambda1UnifiedLS' = result.x[2]
# #     'lambda3UnifiedLS' = result.x[3]
# # }
# best_performance = result.fun
# print("Best MAE: {:.4f}".format(result.fun))
# print("Best Hyperparameters: {}".format(dict(zip(['lr', 'decay'], result.x))))

In [328]:
# # Print the MAE values and hyperparameters for each call
# for i, val in enumerate(result.func_vals):
#     print("Call {}: MAE={:.4f}, Hyperparameters={}".format(i, val, result.x_iters[i]))

In [329]:
# # Print the values of the optimized hyperparameters
# print('Optimized Hyperparameters:')
# for hyperparameter, value in zip(space, result.x):
#     print('{}: {}'.format(hyperparameter.name, value))

## MLP with tuned hyperparameters

In [330]:
torch.manual_seed(43)

In [331]:
mlp = MLP()
  
# Define the loss function and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.03787070596034394, weight_decay=0.009739548876287172)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, verbose=True)

In [332]:
counter = 0
# Run the training loop
while optimizer.param_groups[-1]['lr'] > 1e-4: # 5 epochs at maximum
    
    # Print epoch
#     print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    losses = []
    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
      
      # Get and prepare inputs
#         if counter > 2:
#             break
#         print(counter)
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 100))
        
#         print("Inputs shape: ", inputs.shape)
#         print("Target shape: ", targets.shape)
#         counter += 1
      
      # Zero the gradients
        optimizer.zero_grad()
      
      # Perform forward pass
        outputs = mlp(inputs)
#         print("Output shape: ", outputs.shape)
        
#         outputs_np = outputs.detach().numpy()
#         targets_np = targets.detach().numpy()
        t_V_mt = torch.from_numpy(V_mt.T).float()
    
#         t_vec = targets_np@V_mt.T
#         o_vec = outputs_np@V_mt.T
#         print("t_vec.shape: ", t_vec.shape)
#         print("o_vec.shape: ", o_vec.shape)
        t_vec = torch.matmul(targets, t_V_mt)
        o_vec = torch.matmul(outputs, t_V_mt)
        indicator_matrix = (t_vec >= 0.7)
#         print("indicator matrix: ", indicator_matrix)
#         print(np.all(indicator_matrix == True))
#         t_vec_interacted = t_vec*indicator_matrix
#         o_vec_interacted = o_vec*indicator_matrix
        t_vec_interacted = torch.mul(t_vec, indicator_matrix)
        o_vec_interacted = torch.mul(o_vec, indicator_matrix)
#             t = torch.from_numpy(a)

        # Compute loss
        loss = loss_function(o_vec_interacted, t_vec_interacted)
        
#         print("Loss: ", loss.item())
        print("Epoch : " + str(counter + 1) + " | Error : " + str(loss.item()) + " | LR : " + str(optimizer.param_groups[-1]['lr']))
        losses.append(loss.item())
      # Perform backward pass
        loss.backward()
      
      # Perform optimization
        optimizer.step()
        
        
    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)
    print(f"Loss at epoch {counter + 1} = {mean_loss}")
    if optimizer.param_groups[-1]['lr'] <= 1e-4:
        break;
    counter += 1

# Process is complete.
print('Training process has finished.')

Epoch : 1 | Error : 7.5906476974487305 | LR : 0.03787070596034394
Loss at epoch 1 = 7.5906476974487305
Epoch : 2 | Error : 3.541133165359497 | LR : 0.03787070596034394
Loss at epoch 2 = 3.541133165359497
Epoch : 3 | Error : 2.9373772144317627 | LR : 0.03787070596034394
Loss at epoch 3 = 2.9373772144317627
Epoch : 4 | Error : 3.4786412715911865 | LR : 0.03787070596034394
Loss at epoch 4 = 3.4786412715911865
Epoch : 5 | Error : 2.529480218887329 | LR : 0.03787070596034394
Loss at epoch 5 = 2.529480218887329
Epoch : 6 | Error : 0.9505958557128906 | LR : 0.03787070596034394
Loss at epoch 6 = 0.9505958557128906
Epoch : 7 | Error : 0.2883441746234894 | LR : 0.03787070596034394
Loss at epoch 7 = 0.2883441746234894
Epoch : 8 | Error : 1.391794204711914 | LR : 0.03787070596034394
Loss at epoch 8 = 1.391794204711914
Epoch : 9 | Error : 0.4937231242656708 | LR : 0.03787070596034394
Epoch 00009: reducing learning rate of group 0 to 3.7871e-03.
Loss at epoch 9 = 0.4937231242656708
Epoch : 10 | Erro

In [333]:
pred_X_train = mlp(torch.from_numpy(np.float32(X_train))).detach().numpy()
print(pred_X_train.shape)
cutoff = 3
k = 40
par1 = pred_X_train@V_mt.T
par2 = Y_train@V_mt.T
# mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
# print("Training Metrics")
# print(mae, rmse)

(1694, 100)


In [334]:
mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
Training_Metrics                = ["CDGRS_Train",mae,rmse]
Result_Trn_CDR.loc[len(Result_Trn_CDR)]       = ["Train-3",mae,rmse]

In [335]:
pred_X_test = mlp(torch.from_numpy(np.float32(X_test))).detach().numpy()
print(pred_X_test.shape)
cutoff = 3
k = 40
par1 = pred_X_test@V_mt.T
par1[par1 > 5] = 5
par2 = Y_test
# mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
# print("Test Metrics")
# print(mae, rmse)

(400, 100)


In [336]:
mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
Testing_Metrics                = ["CDGRS_Test",mae,rmse]
Result_Tst_CDR.loc[len(Result_Tst_CDR)]       = ["Test-3",mae,rmse]

In [337]:
Result_Trn_CDR

,Model,MAE,RMSE
0,Train-1,0.753784,0.868620
1,Train-2,0.540425,0.679606
2,Train-3,1.146312,2.053302


In [338]:
Result_Trn_CDR.mean()

C:\Users\NIT\AppData\Local\Temp\ipykernel_14692\3129751611.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Result_Trn_CDR.mean()


MAE     0.813507
RMSE    1.200509
dtype: float64

In [339]:
Result_Tst_CDR

,Model,MAE,RMSE
0,Test-1,1.580367,1.812021
1,Test-2,1.257982,1.527167
2,Test-3,2.665166,3.299865


In [340]:
Result_Tst_CDR.mean()

C:\Users\NIT\AppData\Local\Temp\ipykernel_14692\661790523.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Result_Tst_CDR.mean()


MAE     1.834505
RMSE    2.213018
dtype: float64

In [240]:
# Result_Trn_CDR.drop(index=Result_Trn_CDR.index[-1],axis=0,inplace=True)
# Result_Tst_CDR.drop(index=Result_Tst_CDR.index[-1],axis=0,inplace=True)